In [8]:
!pip install kaggle
!python3 -m pip install pandas

In [14]:
#{"username":"anguis1","key":"8e5aa869bd2b736636371bd2e59902b3"}

import os
import zipfile
import kaggle

os.environ['KAGGLE_USERNAME'] = 'anguis1'
os.environ['KAGGLE_KEY'] = '8e5aa869bd2b736636371bd2e59902b3'

#Nom du dataset
dataset = "vishalsubbiah/pokemon-images-and-types"

#Téléchargement du Dataset
kaggle.api.dataset_download_files(dataset)

with zipfile.ZipFile('pokemon-images-and-types.zip', 'r') as zip_ref:
    zip_ref.extractall('pokemon')

In [10]:
import urllib.request
import json
import pandas as pd

#On charge le fichier csv
df = pd.read_csv('./pokemon/pokemon.csv')

#On crée un dictionnaire associant le nom du pokemon à son image
pokemon_images = {}
for filename in os.listdir("./pokemon/images/images"):
    if filename.endswith(".png"):
        name = filename.split(".")[0]
        pokemon_images[name] = os.path.join("pokemon/images/images", filename)

# Ajouter une colonne "image" au dataframe
df["image"] = df["Name"].apply(lambda x: pokemon_images.get(x, None))

# Supprimer les lignes sans image
df = df.dropna(subset=["image"])

# Afficher le dataframe
print(df.head())

         Name  Type1   Type2                                 image
0   bulbasaur  Grass  Poison   pokemon/images/images/bulbasaur.png
1     ivysaur  Grass  Poison     pokemon/images/images/ivysaur.png
2    venusaur  Grass  Poison    pokemon/images/images/venusaur.png
3  charmander   Fire     NaN  pokemon/images/images/charmander.png
4  charmeleon   Fire     NaN  pokemon/images/images/charmeleon.png


In [29]:
import numpy as np
from sklearn.cluster import KMeans

def get_dominant_colors(image_path, n_colors):
    # Charger l'image et la convertir en une matrice numpy
    img = Image.open(image_path)
    img = np.array(img)

    # Obtenir un tableau de pixels en deux dimensions
    pixels = img.reshape(-1, 3)
    #-1 car l'argument s'adapte à la taille de l'image et 3 pour le RGB
    
    # Créer un objet de clustering
    kmeans = KMeans(n_clusters=n_colors)

    # Entraîner le modèle
    kmeans.fit(pixels)

    # Récupérer les couleurs dominantes
    dominant_colors = kmeans.cluster_centers_

    # Convertir les couleurs en entiers et les ajouter à une liste
    dominant_colors = [tuple(map(int, color)) for color in dominant_colors]
    
    return '#{:02x}{:02x}{:02x}'.format(dominant_colors)

In [30]:
# import random
import random

def satisfied_or_not():
    
    #Fonction pour générer un avis aléatoire (positif ou négatif) de l'utilisateur pour pouvoir effectuer des tests
    
    return random.choice(['Yes', 'No'])

In [ ]:
import os
import csv
import json
from PIL import Image

# Chemin du dossier contenant les images
folder_path = './pokemon/images/images'

# Liste des extensions de fichier que nous voulons inclure dans l'analyse
extensions = ['.jpg', '.jpeg', '.png']

# Initialisation de la liste pour stocker les informations de chaque image
list_images_info = []

# Ouverture du fichier CSV
with open('./pokemon/pokemon.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader) # Ignorer la première ligne (en-tête)
    for row in csv_reader:
        # Récupère les informations du CSV
        pokemon_name = row[0]
        type1 = row[1]
        try :
            type2 = row[2]
        except Exception as e:
            type2 = 'None'
            continue

        filename = row[0]+'.png'
    
        try:
            # Vérifie si le fichier est une image avec une extension spécifiée
            if any(filename.lower().endswith(ext) for ext in extensions):
                # Chemin complet de l'image
                filepath = os.path.join(folder_path, filename)
                # Ouvre l'image avec PIL
                with Image.open(filepath) as img:
                    # Stocke les informations dans un dictionnaire
                    images_info = {'filename': filename,
                                   'pokemon_name': pokemon_name,
                                   'type1': type1,
                                   'type2': type2,
                                   'size': os.path.getsize(filepath),
                                   'k-means_regions': get_dominant_colors(filepath,1),
                                   'favorite_utilisateur1': satisfied_or_not(),
                                   'favorite_utilisateur2': satisfied_or_not(),
                                   'favorite_utilisateur3': satisfied_or_not(),
                                   'favorite_utilisateur4': satisfied_or_not()}
                    # Ajoute le dictionnaire à la liste
                    list_images_info.append(images_info)
        except Exception as e:
            filename = row[0]+'.jpg' #Si le fichier n'existe pas en .png, on teste de le trouver en .jpg
    
            try:
                # Vérifie si le fichier est une image avec une extension spécifiée
                if any(filename.lower().endswith(ext) for ext in extensions):
                    # Chemin complet de l'image
                    filepath = os.path.join(folder_path, filename)
                    # Ouvre l'image avec PIL
                    with Image.open(filepath) as img:
                        # Stocke les informations dans un dictionnaire
                        images_info = {'filename': filename,
                                       'pokemon_name': pokemon_name,
                                       'type1': type1,
                                       'type2': type2,
                                       'size': os.path.getsize(filepath),
                                       'k-means_regions': get_dominant_colors(filepath,1),
                                       'favorite_utilisateur1': satisfied_or_not(),
                                       'favorite_utilisateur2': satisfied_or_not(),
                                       'favorite_utilisateur3': satisfied_or_not(),
                                       'favorite_utilisateur4': satisfied_or_not()}
                        # Ajoute le dictionnaire à la liste
                        list_images_info.append(images_info)
            except Exception as e:
                print(f"Erreur lors de l'extraction des informations de {row[0]}: {e}")
                continue

# Écriture du résultat dans un fichier JSON
with open('images_info.json', 'w') as json_file:
    json.dump(list_images_info, json_file)

Erreur lors de l'extraction des informations de bulbasaur: [Errno 2] No such file or directory: './pokemon/images/images/bulbasaur.jpg'
Erreur lors de l'extraction des informations de ivysaur: [Errno 2] No such file or directory: './pokemon/images/images/ivysaur.jpg'
Erreur lors de l'extraction des informations de venusaur: [Errno 2] No such file or directory: './pokemon/images/images/venusaur.jpg'
Erreur lors de l'extraction des informations de charizard: [Errno 2] No such file or directory: './pokemon/images/images/charizard.jpg'
Erreur lors de l'extraction des informations de butterfree: [Errno 2] No such file or directory: './pokemon/images/images/butterfree.jpg'
Erreur lors de l'extraction des informations de weedle: [Errno 2] No such file or directory: './pokemon/images/images/weedle.jpg'
Erreur lors de l'extraction des informations de kakuna: [Errno 2] No such file or directory: './pokemon/images/images/kakuna.jpg'
Erreur lors de l'extraction des informations de beedrill: [Errno

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


Erreur lors de l'extraction des informations de gloom: [Errno 2] No such file or directory: './pokemon/images/images/gloom.jpg'
Erreur lors de l'extraction des informations de vileplume: [Errno 2] No such file or directory: './pokemon/images/images/vileplume.jpg'
Erreur lors de l'extraction des informations de paras: [Errno 2] No such file or directory: './pokemon/images/images/paras.jpg'
Erreur lors de l'extraction des informations de parasect: [Errno 2] No such file or directory: './pokemon/images/images/parasect.jpg'
Erreur lors de l'extraction des informations de venonat: [Errno 2] No such file or directory: './pokemon/images/images/venonat.jpg'
Erreur lors de l'extraction des informations de venomoth: [Errno 2] No such file or directory: './pokemon/images/images/venomoth.jpg'
Erreur lors de l'extraction des informations de poliwrath: [Errno 2] No such file or directory: './pokemon/images/images/poliwrath.jpg'
Erreur lors de l'extraction des informations de bellsprout: [Errno 2] No